In [53]:
import sys, os, io, json, numpy as np, random, time
#from sklearn.metrics import roc_auc_score
from importlib import reload
import extra_utils
reload(extra_utils)
from extra_utils import time_features, subpool, test_extra_predictions
import forest
reload(forest)
from forest import build_tree, EMatrix
import xgboost as xgb
import tensorflow as tf
import split
reload(split)
from split import SplitMaker
#from split import make_split_quick, make_gax, split_quick_graph, split_quick_graph_transpose
import graphviz
from sklearn import metrics

In [54]:
split_maker_old = SplitMaker.make_split_old()

In [55]:
pool = np.load('../prepare_dataset/pool_00.npz')
features_bool, f_time, label_orig = (pool[name] for name in ['features', 'f_time', 'label'])
label = label_orig

In [56]:
np.random.seed(42)
features_orig = np.random.normal(loc=features_bool*1.0, scale=0.01)
#features = features[:150000,:]; label = label_orig[:150000, :]

f_test, l_test = features_orig[150000:,:], label_orig[150000:, :]
features = features_orig[:150000,:]; label = label_orig[:150000, :]

### Тестирование инвариантности сплита относительно замены местами столбцов фичей

```
features_swap = np.concatenate([features[:,7:8], features[:,1:7], features[:,0:1], features[:,8:]], axis=1)

%%time
split_maker_old.split(np.zeros(label.shape), features, label)
#make_split(np.zeros(label.shape), features, label)

%%time
split_maker_old.split(np.zeros(label.shape), features_swap, label)
#make_split(np.zeros(label.shape), features_swap, label)
```

In [57]:
features.shape, label.shape

((150000, 15), (150000, 1))

In [58]:
global_ax = split_maker_old.make_gax(features)
featuresT = features.T
global_axT = global_ax.T

In [59]:
split_maker = SplitMaker.make_split_new(reduce_axis=0, make_transpose=True)

In [60]:
%%time
start_params = {'max_depth': 8, 'learning_rate':0.3, 'transposed_feature1111': 1, 'unbalanced_penalty':0.001}
tf.constant
forest.time1 = forest.time2 =0
forest.time3 = []
with tf.Session(graph=split_maker.graph) as s:
    tree = build_tree(start_params, EMatrix(featuresT if split_maker.reduce_axis==1 else features, 
                                            label, np.zeros(label.shape), 
                                            gax=global_axT if split_maker.reduce_axis==1 else global_ax, 
                                            splitgax=True), split_maker=split_maker, sess=s)
print("\nsplit time = {:.5f}, {:.5f}, [{}]".format(forest.time1, forest.time2, ", ".join("{:.5f}".format(tt) for tt in forest.time3)), file=sys.stderr)
tree_arrays = forest.init_arrays(tree, forest.init_id(tree))

1  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  1  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  2  3  4  5  5  4  5  5  3  4  5  5  

CPU times: user 6.46 s, sys: 257 ms, total: 6.71 s
Wall time: 2.5 s


4  5  5  
split time = 6.55365, 0.11795, [0.58596, 0.25116, 0.13560, 0.07351, 0.04669, 0.03291, 0.02687, 0.02711, 0.01508, 0.01744, 0.01889, 0.01417, 0.02827, 0.02848, 0.02630, 0.01662, 0.01577, 0.01514, 0.01591, 0.04806, 0.03592, 0.02465, 0.02180, 0.01871, 0.02433, 0.02567, 0.01558, 0.02477, 0.02339, 0.01980, 0.01852, 0.01805, 0.01651, 0.01607, 0.07438, 0.07061, 0.03405, 0.02698, 0.01675, 0.02351, 0.02901, 0.02172, 0.02158, 0.03850, 0.03315, 0.02271, 0.02419, 0.02087, 0.02068, 0.01780, 0.02369, 0.02280, 0.02060, 0.01844, 0.01822, 0.01840, 0.01749, 0.01637, 0.01742, 0.01322, 0.01485, 0.01506, 0.01680, 0.01499, 0.01718, 0.14548, 0.10947, 0.10134, 0.05596, 0.03497, 0.02457, 0.02242, 0.03180, 0.02640, 0.01979, 0.05634, 0.05138, 0.02558, 0.03092, 0.02421, 0.01918, 0.01951, 0.02813, 0.02514, 0.02531, 0.02180, 0.01760, 0.01596, 0.01702, 0.01828, 0.01996, 0.01607, 0.01475, 0.01600, 0.01599, 0.01724, 0.01600, 0.04095, 0.03849, 0.03333, 0.03359, 0.03432, 0.01738, 0.01870, 0.01606, 0.01819, 0.01

In [61]:
import pstats
p = pstats.Stats('build_tree.prof')
#p.sort_stats('tottime').print_stats()

In [62]:
pred = forest.tree_apply(tree_arrays, features)
pred_test = forest.tree_apply(tree_arrays, f_test)
auc1l = metrics.roc_auc_score(label[:, 0], pred)
auc1t = metrics.roc_auc_score(label[:, 0], pred_test)
auc0 = metrics.roc_auc_score(label[:, 0], np.zeros(label.shape[0]))
auc1l, auc1t, auc0

(0.564954307935, 0.5019091667880746, 0.5)

In [63]:
#tree.left.left.left.left.val, tree.val
tree.left.left

In [64]:
#forest.tree2gv(tree)

In [65]:
%%time
boost_params = {'max_depth': 4, 'learning_rate':0.3, 'splitgax': True, 'transposed_feature': 0, 'unbalanced_penalty':0.001}
booster = forest.train(boost_params, EMatrix(featuresT if boost_params['transposed_feature'] else features, label), num_boost_round=10)


0 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
1 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
2 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
3 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
4 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
5 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
6 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
7 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
8 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
9 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  

CPU times: user 17.6 s, sys: 76.9 ms, total: 17.7 s
Wall time: 6.85 s


In [66]:
####################################################

In [67]:
#%%time
#gax13 = make_gax(features)
#print(gax13.shape)

In [68]:
pred2 = booster.predict(features)
pred2b = booster.predict(features, tree_limit=10)
pred_test = booster.predict(f_test)
main_auc_l = metrics.roc_auc_score(label[:, 0], pred2)
main_auc_t = metrics.roc_auc_score(l_test[:, 0], pred_test)
main_auc_l, main_auc_t

(0.5605287476686079, 0.5402890562386409)

In [17]:
0.5783850422568331,0.547297544877569

(0.5783850422568331, 0.547297544877569)

In [18]:
#np.min(pred2b), np.max(pred2b)

In [19]:
tree2gv(booster.forest[45][0])

NameError: name 'tree2gv' is not defined

In [20]:
dmat = xgb.DMatrix(features, label)

In [22]:
%%time
params = {'max_depth': 4, 'objective': 'binary:logistic', 'eta': 0.3, 'tree_method': 'exact', 'lambda': 1}
model = xgb.train(params, dmat, num_boost_round=10)

[23:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[23:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:00:48] /work

In [23]:
dmat_test = xgb.DMatrix(f_test, l_test)

In [24]:
pred_fast = model.predict(dmat)
pred_test = model.predict(dmat_test)

metrics.roc_auc_score(label[:, 0], pred_fast), metrics.roc_auc_score(l_test[:, 0], pred_test)

(0.564382620107551, 0.5476151803114241)

In [ ]:
pred_fast = model.predict(dmat, ntree_limit=35)
metrics.roc_auc_score(label[:, 0], pred_fast)

In [ ]:
pred2b = booster.predict(features, tree_limit=15)
metrics.roc_auc_score(label[:, 0], pred2b)

In [ ]:
0.5987831226886691, 0.5457320402572653

In [ ]:
qq = -np.log(1/pred_fast-1) 
np.min(qq), np.max(qq)

In [ ]:
diff = np.abs(qq - pred2b)
np.mean(diff > 0.1)

In [ ]:
model.dump_model('xgb.dump')

In [ ]:
tree2gv(booster.forest[1][0])

In [ ]:
!cat xgb.dump | tail -n+$(echo 39*31+31 | bc) | head -n 100

### Extra begins here

In [32]:
extra_f = time_features(f_time)

In [33]:
extra_f.shape

(300000, 2)

In [43]:
f_train, extra_f_train, label_train = subpool((features_orig, extra_f, label_orig), f_time, (0, 0.2))

In [44]:
f_train.shape, extra_f_train.shape, label_train.shape

((59816, 15), (59816, 2), (59816, 1))

In [45]:
%%time
boost_params = {'max_depth': 4, 'learning_rate':0.1, 'splitgax': True, 'transposed_feature': 0, 'unbalanced_penalty':0.001}
booster = forest.train(boost_params, EMatrix(f_train.T if boost_params['transposed_feature'] else f_train, label_train), num_boost_round=50)


0 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
1 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
2 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
3 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
4 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
5 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
6 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
7 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
8 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
9 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
10 round


CPU times: user 42.6 s, sys: 2.2 s, total: 44.8 s
Wall time: 17.7 s


2  3  4  4  3  4  4  2  3  4  4  3  4  4  

In [46]:
whole_prediction = booster.predict(features_orig)

In [50]:
test_extra_predictions(20, f_time, label_orig, whole_prediction)

(0.0, 0.05) score = 0.7474347329785658
(0.05, 0.1) score = 0.7136549630003847
(0.1, 0.15) score = 0.6905347190056565
(0.15, 0.2) score = 0.6742037678268004
(0.2, 0.25) score = 0.6189839539404536
(0.25, 0.3) score = 0.6130939220466572
(0.3, 0.35) score = 0.5822061545827396
(0.35, 0.4) score = 0.5499274195800611
(0.4, 0.45) score = 0.5434790274825424
(0.45, 0.5) score = 0.5186325268988002
(0.5, 0.55) score = 0.4874239058772766
(0.55, 0.6) score = 0.4641952642235025
(0.6, 0.65) score = 0.43741246201696643
(0.65, 0.7) score = 0.41581390645412536
(0.7, 0.75) score = 0.3783613660486673
(0.75, 0.8) score = 0.34287195722266955
(0.8, 0.85) score = 0.31103091450570697
(0.85, 0.9) score = 0.27720118942703326
(0.9, 0.95) score = 0.24198711458066374
(0.95, 1.0) score = 0.1948191703750582
